In [24]:
import sqlite3
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [25]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [26]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [27]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [28]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [29]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [30]:
pd.read_sql(
    """
    select
    sum(case when CRIM is null then 1 else 0 end) CRIM,
    sum(case when ZN is null then 1 else 0 end) ZN,
    sum(case when INDUS is null then 1 else 0 end) INDUS,
    sum(case when CHAS is null then 1 else 0 end) CHAS,
    sum(case when NOX is null then 1 else 0 end) NOX
    from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [31]:
pd.read_sql(
    """
    select count(distinct CRIM) uCRIM,
    count(distinct ZN) uZN,
    count(distinct INDUS) uINDUS,
    count(distinct CHAS) uCHAS,
    count(distinct NOX) uNOX FROM boston
    """,
    con,
)

,uCRIM,uZN,uINDUS,uCHAS,uNOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [32]:
pd.read_sql(
    """
    with
        CRIM as (select 'CRIM' as col, CRIM_med = min(CRIM) as res from(
        select  CRIM as CRIM_med
        from boston
        order by CRIM
        limit 1
        offset (select count(*) from boston) / 2),
        boston),

        ZN as (select 'ZN' as col, ZN_med = min(ZN) as res from(
        select ZN as ZN_med
        from boston
        order by ZN
        limit 1
        offset (select count(*) from boston) / 2),
        boston),

        INDUS as (select 'INDUS' as col, INDUS_med = min(INDUS) as res from(
        select INDUS as INDUS_med
        from boston
        order by INDUS
        limit 1
        offset (select count(*) from boston) / 2),
        boston),

        CHAS as (select 'CHAS' as col, CHAS_med = min(CHAS) as res from(
        select CHAS as CHAS_med
        from boston
        order by CHAS
        limit 1
        offset (select count(*) from boston) / 2),
        boston),

        NOX as (select 'NOX' as col, NOX_med = min(NOX) as res from(
        select NOX as NOX_med
        from boston
        order by NOX
        limit 1
        offset (select count(*) from boston) / 2),
        boston)

        select col from (
        select * from CRIM
        union
        select * from ZN
        union
        select * from INDUS
        union
        select * from CHAS
        union
        select * from NOX
        )
        where res = 1

    """,
    con,
)

,col
0,CHAS
1,ZN


Вывод:признак CHAS - бинарный, для него свойство равенства медианы и минимального значения очевидно. Для признака ZN >= половины наблюдений дают одно и то же значение, что говорит о неравномерности распределения значений признака

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [33]:
pd.read_sql(
    """
    with
        h as
            (select avg(RM) as avg_high
            from boston
            group by MEDV
            order by MEDV desc
            limit 1),

        n25 as
                (select 25 as n_h,
                avg(rm) as rm_avg,
                avg_high as rm_avg_h,
                avg_high-avg(RM) as diff
            from
                (select RM, MEDV
                from boston
                order by MEDV
                limit 25), h),

        n50 as
                (select 50 as n_h,
                avg(rm) as rm_avg,
                avg_high as rm_avg_h,
                avg_high-avg(RM) as diff
            from
                (select RM, MEDV
                from boston
                order by MEDV
                limit 50), h),

        n100 as
                (select 100 as n_h,
                avg(rm) as rm_avg,
                avg_high as rm_avg_h,
                avg_high-avg(RM) as diff
            from
                (select RM, MEDV
                from boston
                order by MEDV
                limit 100), h),
        n200 as
                (select 200 as n_h,
                avg(rm) as rm_avg,
                avg_high as rm_avg_h,
                avg_high-avg(RM) as diff
            from
                (select RM, MEDV
                from boston
                order by MEDV
                limit 200), h),
        n300 as
                (select 300 as n_h,
                avg(rm) as rm_avg,
                avg_high as rm_avg_h,
                avg_high-avg(RM) as diff
            from
                (select RM, MEDV
                from boston
                order by MEDV
                limit 300), h)

    select * from n25
    union
    select * from n50
    union
    select * from n100
    union
    select * from n200
    union
    select * from n300

    """,
    con,
)

,n_h,rm_avg,rm_avg_h,diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


Вывод: влияет, имеет место положительная корреляция

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [34]:
pd.read_sql(
    """
    with T AS
        (select
            LSTAT,
            MEDV,
            rank () over ( 
            order by LSTAT 
            ) RANK
        from boston)
    select 
        LSTAT,
        MEDV,
        RANK
    from T
    where MEDV = (select max(MEDV) from boston)
    """,
    con,
)

,LSTAT,MEDV,RANK
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Вывод: подтверждается предположение, что в самых дорогих домах доля населения с низким статусом мала.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [35]:
pd.read_sql(
    """
    select CHAS, avg(MEDV) as avgMEDV from boston group by CHAS
    """,
    con,
)

,CHAS,avgMEDV
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [36]:
pd.read_sql(
    """
    select col from
    (select
        'CRIM' AS col,
        (select avg(CRIM) from boston where chas>0)>avg(CRIM) as result
    from boston
    union
    select
        'ZN' AS col,
        (select avg(ZN) from boston where chas>0)>avg(ZN) as result
    from boston
    union
    select
        'INDUS' AS col,
        (select avg(INDUS) from boston where chas>0)>avg(INDUS) as result
    from boston
    union
    select
        'NOX' AS col,
        (select avg(NOX) from boston where chas>0)>avg(NOX) as result
    from boston)
    where result=1
    """,
    con,
)

,col
0,INDUS
1,NOX


Вывод: INDUS: промышленные предприятия используют водные ресурсы;NOX: увеличение концентрации оксидов азота связано с пром.зоной, увеличением INDUS; CRIM меньше в пром. зоне

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [37]:
pd.read_sql(
    """
    select 10 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*1/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*1/10) AS NOX
    union
    select 20 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*2/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*2/10) AS NOX
    union
    select 30 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*3/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*3/10) AS NOX
    union
    select 40 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*4/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*4/10) AS NOX
    union
    select 50 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*5/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*5/10) AS NOX
    union
    select 60 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*6/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*6/10) AS NOX
    union
    select 70 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*7/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*7/10) AS NOX
    union
    select 80 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*8/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*8/10) AS NOX
    union
    select 90 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*9/10) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*9/10) AS NOX
    union
    select
        100 as percentile,
        (select INDUS from boston order by INDUS  limit 1 offset (select count() from boston)*10/10-1) AS INDUS,
        (select NOX from boston order by NOX  limit 1 offset (select count() from boston)*10/10-1) AS NOX
    """,
    con,
)

,percentile,INDUS,NOX
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,12.83,0.575
6,70,18.10,0.605
7,80,18.10,0.668
8,90,19.58,0.713
9,100,27.74,0.871


Вывод: есть корреляция, связь прослеживается.